# Wikipedia全文をCSVに変換

種々の検証において再利用を可能とするため、Wikipedia全文をCSVファイルに変換しておきます。

## (1) Wikipediaコンテンツファイルから本文を抽出

### (1-1) Wikipediaコンテンツ取得

Wikipedia コンテンツの一覧を、以下のURLで参照します。

https://dumps.wikimedia.org/jawiki/latest/

このページのリストから、以下のファイルのリンクをクリックしてダウンロードします。

2.4GBほどになります。

```
jawiki-latest-pages-articles.xml.bz2    21-May-2017 01:07    2492605537
```

### (1-2) Wikipedia Extractor（本文抽出ツール）の取得

下記サイト内のDownloadsからWikipedia ExtractorのPythonスクリプトを直接ダウンロードします。

http://medialab.di.unipi.it/wiki/Wikipedia_Extractor

### (1-3) Wikipediaコンテンツファイルから本文を抽出

下記コマンドで抽出を実行します。

python3 WikiExtractor.py -b 10M -o extracted jawiki-latest-pages-articles.xml.bz2

これで、extractedディレクトリ下に、10MB毎に区切られた抽出後ファイルが生成されていきます。

extractedディレクトリ配下には、AA、AB、AC・・・といったサブディレクトリが生成されます。

サブディレクトリごとに、１００本のファイルが生成されます。

（すなわちサブディレクトリごとに1GBまで）

実行には４０分ほどかかるとのことでしたが、実績としては、5/22 13:51開始〜14:26終了、なので３５分ほどで完了しています。

実行例：
```
MacBookPro-makmorit-jp:Wikipedia makmorit$ pwd
/Users/makmorit/Documents/Development/Wikipedia
MacBookPro-makmorit-jp:Wikipedia makmorit$ ls -al
total 4868576
drwxr-xr-x   4 makmorit  staff         136 May 22 13:44 .
drwxr-xr-x  10 makmorit  staff         340 May 22 13:43 ..
-rw-r--r--@  1 makmorit  staff       98379 Mar 23  2016 WikiExtractor.py
-rw-r--r--@  1 makmorit  staff  2492605537 May 22 13:27 jawiki-latest-pages-articles.xml.bz2
MacBookPro-makmorit-jp:Wikipedia makmorit$ python WikiExtractor.py -b 10M -o extracted jawiki-latest-pages-articles.xml.bz2
INFO: Loaded 0 templates in 0.0s
INFO: Starting page extraction from jawiki-latest-pages-articles.xml.bz2.
INFO: Using 7 extract processes.
WARNING: Template errors in article '自然数' (4671): title(9) recursion(0, 0, 0)
WARNING: Template errors in article '階乗' (12763): title(1) recursion(0, 0, 0)
INFO: Extracted 10000 articles (225.3 art/s)
INFO: Extracted 20000 articles (263.3 art/s)
（中略）
WARNING: Template errors in article 'プロ経営者' (3516673): title(2) recursion(0, 0, 0)
INFO: Extracted 1410000 articles (990.3 art/s)
WARNING: Template errors in article '2017年のWTAツアー' (3535919): title(2) recursion(0, 0, 0)
INFO: Extracted 1420000 articles (995.9 art/s)
INFO: Extracted 1430000 articles (910.8 art/s)
WARNING: Template errors in article 'Wikipedia:削除依頼/性液' (3583405): title(1) recursion(0, 0, 0)
INFO: Extracted 1440000 articles (986.9 art/s)
INFO: Finished 7-process extraction of 1446777 articles in 2059.6s (702.5 art/s)
MacBookPro-makmorit-jp:Wikipedia makmorit$ date
Mon May 22 14:25:52 JST 2017
MacBookPro-makmorit-jp:Wikipedia makmorit$ 
```

## (2) Wikipedia全文のCSVファイルを生成

### (2-1) 抽出コンテンツの内容についての注意

下記のように、タイトルに : 文字が含まれている場合は有効な記事でないようです。

```
<doc id="1" url="https://ja.wikipedia.org/wiki?curid=1" title="Wikipedia:アップロードログ 2004年4月">
Wikipedia:アップロードログ 2004年4月
<ul>
</doc>
```

したがってTaggedDocumentへの変換時は、これらのエントリーは除外する必要があります。

### (2-2) 抽出コンテンツをCSVへ変換

有効なドキュメントは下記のようになっています。

```
<doc id="2466881" url="https://ja.wikipedia.org/wiki?curid=2466881" title="ヒントン駅">
ヒントン駅
ヒントン駅（ヒントンえき、英語：Hinton Station）は、ウェストバージニア州 メープル・アベニューのセカンド・アヴェニュー100にある駅。 昔のチェサピーク・アンド・オハイオ鉄道の駅である。
アムトラックの停車する列車は下記の通り。
</doc>
```

一旦、pandasで処理（再利用）できる様に、ヘッダー付きのCSVファイルに格納しておきます。

In [1]:
'''
    環境準備
'''
import sys
import os
import re
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)
if learning_dir not in sys.path:
    sys.path.append(learning_dir)

In [2]:
def process_start_tag(line):
    '''
        docタグ開始と判定した場合
        docタグから取得したid, titleを戻す。

        ただし、titleに':' '曖昧さ回避'が
        含まれている場合は無効エントリーとし、
        docタグ開始と判定しない
    '''
    tag_start_pattern = r'<doc id="([0-9]+)" url=".+" title="([^:]+)">'

    sm = re.match(tag_start_pattern, line)
    if sm is None:
        return None, None
        
    try:
        id = sm.group(1)
        title = sm.group(2)
        
        if title.find('曖昧さ回避') > 0:
            id = None
            title = None

    except:
        id = None
        title = None
        pass

    return id, title

In [3]:
def process_end_tag(line):
    '''
        docタグの終了判定
    '''
    tag_end_pattern = r'</doc>'

    em = re.match(tag_end_pattern, line)
    if em is not None:
        return True

    return False

In [4]:
def parse_wiki_xml(lines):
    '''
        Wikipediaから抽出されたコンテンツをリスト化する。
        
        Wikipediaから抽出されたコンテンツはXML形式になっているが、
        通常Pythonで用意されているパーサーでは処理不可能なので、
        この関数でparseを行う
    '''
    parsed_wiki = []
    tag_started = False

    for line in lines:
        line = line.replace('\n', '')

        if not tag_started:
            '''
                doc タグの開始判定
            '''
            id, title = process_start_tag(line)
            if id is not None:
                tag_started = True
                content = ''
            else:
                tag_started = False
            continue

        if not tag_started:
            continue

        if process_end_tag(line):
            '''
                doc タグが終了したら、
                id, title, 本文をリストに出力
                ただし、本文が25文字以下なら
                書きかけコンテンツとみなして無視
            '''
            if len(content) > 25:
                parsed_wiki.append([int(id), title, content])
            tag_started = False
            content = ''
        else:
            '''
                doc タグの途中なら本文として扱う
                ただし先頭行はタイトルなので
                本文として扱わない
            '''
            if line != title:
                content += line
    
    return parsed_wiki

In [5]:
def parse_wiki_xml_file(file_path):
    try:
        with open(file_path) as wikifile:
            lines = wikifile.readlines()
            wikifile.close()
            parsed_wiki = parse_wiki_xml(lines)
    except:
        parsed_wiki = []
        pass

    return parsed_wiki

In [6]:
def get_wiki_document_list(extracted_dir_path):
    '''
        Wikipedia全文ファイル群からCSVを生成
    '''
    wiki_document_list = []

    for prefix in ['AA', 'AB', 'AC']:
        for file_no in range(100):
            file_path = os.path.join(extracted_dir_path, prefix, 'wiki_%02d' % file_no)
            parsed_wiki = parse_wiki_xml_file(file_path)
            wiki_document_list.extend(parsed_wiki)

    return wiki_document_list

In [7]:
'''
    Wikipedia全文から、データフレームを生成し、
    CSVファイルとして出力する
'''
import pandas as pd
 
extracted_dir_path = '/Users/makmorit/Documents/Development/Wikipedia/extracted'

wiki_document_list = get_wiki_document_list(extracted_dir_path)
df = pd.DataFrame(wiki_document_list, columns=['id', 'title', 'content'])

csv_file_name = os.path.join(extracted_dir_path, 'Wikipedia-content.csv')
df.to_csv(csv_file_name)

### (2-3) 抽出コンテンツをCSVから読込

Wikipedia全文のCSVファイルからデータフレームを読み込めるかどうか確認します。

In [8]:
import numpy as np

loaded_df = pd.read_csv(csv_file_name)

wiki_ids = np.array(loaded_df['id'])
wiki_titles = np.array(loaded_df['title'])
wiki_contents = np.array(loaded_df['content'])

In [16]:
loaded_df.shape

(1006163, 4)

In [17]:
wiki_ids

array([      5,      10,      11, ..., 3600297, 3600299, 3600310])

In [18]:
wiki_titles

array(['アンパサンド', '言語', '日本語', ..., '豊橋市議会', '高慢と偏見とゾンビ', '台産'], dtype=object)

In [19]:
wiki_contents[0]

'アンパサンド (, &) とは「…と…」を意味する記号である。英語の に相当するラテン語の の合字で、 (et cetera = and so forth)を と記述することがあるのはそのため。Trebuchet MSフォントでは、と表示され "et" の合字であることが容易にわかる。その使用は1世紀に遡ることができ (1)、5世紀中葉 (2,3) から現代 (4-6) に至るまでの変遷がわかる。Z に続くラテン文字アルファベットの27字目とされた時期もある。アンパサンドと同じ役割を果たす文字に「のet」と呼ばれる、数字の「7」に似た記号があった(, U+204A)。この記号は現在もゲール文字で使われている。記号名の「アンパサンド」は、ラテン語まじりの英語「& はそれ自身 "and" を表す」(& per se and) のくずれた形である。英語以外の言語での名称は多様である。日常的な手書きの場合、欧米でアンパサンドは「ε」に縦線を引く単純化されたものが使われることがある。また同様に、「t」または「+（プラス）」に輪を重ねたような、無声歯茎側面摩擦音を示す発音記号「」のようなものが使われることもある。プログラミング言語では、C など多数の言語で AND 演算子として用いられる。以下は C の例。PHPでは、変数宣言記号（$）の直前に記述することで、参照渡しを行うことができる。BASIC 系列の言語では文字列の連結演算子として使用される。codice_4 は codice_5 を返す。また、主にマイクロソフト系では整数の十六進表記に codice_6 を用い、codice_7 （十進で15）のように表現する。SGML、XML、HTMLでは、アンパサンドを使ってSGML実体を参照する。'